# ConvTranspose2d() 이해

ConvTranspose2d() 함수는 Conv2d() 함수가 수행하는 과정을 역으로 수행하는 계산을 한다.<br>
출처: <a href = "https://cumulu-s.tistory.com/m/29">https://cumulu-s.tistory.com/m/29</a><br>
참고: <a href = "https://simonjisu.github.io/deeplearning/2019/10/27/convtranspose2d.html">https://simonjisu.github.io/deeplearning/2019/10/27/convtranspose2d.html</a>

In [6]:
import torch
import torch.nn as nn
import numpy as np

In [2]:
# sample values.
test_input = torch.Tensor([[[[1, 2, 3], [4, 5, 6]]]])
print("input size: ", test_input.shape)
print("test input: ", test_input)

input size:  torch.Size([1, 1, 2, 3])
test input:  tensor([[[[1., 2., 3.],
          [4., 5., 6.]]]])


In [4]:
# sample model. It has nn.ConvTranspose2d(1, 3, 4, 1, 0, bias = False)
# First parameter = Channels of input (=1)
# Second parameter = Channels of output (=3)
# Third parameter = Kernel size (=4)
# Fourth parameter = stride (=1)
# fifth parameter = padding (=0)

class sample(nn.Module):
  def __init__(self):
    super(sample, self).__init__()
    self.main = nn.ConvTranspose2d(1, 3, 4, 1, 0, bias = False)

  def forward(self, input):
    return self.main(input)

In [5]:
Model = sample()

# Print model's original parameters.
for name, param in Model.state_dict().items():
  print("name: ", name)
  print("Param: ", param)
  print("Param shape: ", param.shape)

name:  main.weight
Param:  tensor([[[[ 4.6074e-02, -5.1549e-02, -5.5780e-02, -1.0099e-01],
          [ 9.5265e-02, -1.2498e-01, -1.0886e-01, -2.9084e-02],
          [-6.7465e-02, -4.5601e-02,  4.5032e-02, -3.6459e-03],
          [ 8.9165e-02, -9.0356e-02, -1.3539e-01, -1.2684e-01]],

         [[-3.2841e-02, -8.2418e-05,  3.7581e-02,  6.7136e-02],
          [-9.8265e-02,  6.9357e-02,  1.1041e-01, -3.3170e-02],
          [-6.0295e-02, -1.6185e-02,  5.7947e-02, -5.9235e-02],
          [ 1.0360e-01,  3.6945e-02, -7.2067e-02, -6.5677e-02]],

         [[-9.5370e-02, -6.8341e-02, -2.0472e-02,  8.5757e-02],
          [ 4.2107e-03, -6.2693e-02, -3.6892e-02, -7.7881e-02],
          [ 7.3321e-03, -1.5210e-02, -7.7257e-02, -9.8794e-02],
          [ 1.9162e-02, -7.7969e-03,  9.6016e-02,  1.4350e-02]]]])
Param shape:  torch.Size([1, 3, 4, 4])


In [8]:
# I makes 48 values from 0.1 to 4.8 and make (1, 3, 4, 4) shape
np_sam = np.linspace(0.1, 4.8, num = 48)
np_sam_torch = torch.Tensor(np_sam)
sam_tor = np_sam_torch.view(1, 3, 4, 4)

In [11]:
# Modify model's parameters using 4 for loops.
with torch.no_grad():
    # batch:1, channel:3, width:4, height:4 대입됨
    batch, channel, width, height = Model.main.weight.shape 
    # 구분하기 쉬운 데이터를 모델의 가중치에 대입
    for b in range(batch):
        for c in range(channel):
            for w in range(width):
                for h in range(height):
                    Model.main.weight[b][c][w][h] = sam_tor[b][c][w][h]

# Check parameter modification.
print("Model weight: ", Model.main.weight)

Model weight:  Parameter containing:
tensor([[[[0.1000, 0.2000, 0.3000, 0.4000],
          [0.5000, 0.6000, 0.7000, 0.8000],
          [0.9000, 1.0000, 1.1000, 1.2000],
          [1.3000, 1.4000, 1.5000, 1.6000]],

         [[1.7000, 1.8000, 1.9000, 2.0000],
          [2.1000, 2.2000, 2.3000, 2.4000],
          [2.5000, 2.6000, 2.7000, 2.8000],
          [2.9000, 3.0000, 3.1000, 3.2000]],

         [[3.3000, 3.4000, 3.5000, 3.6000],
          [3.7000, 3.8000, 3.9000, 4.0000],
          [4.1000, 4.2000, 4.3000, 4.4000],
          [4.5000, 4.6000, 4.7000, 4.8000]]]], requires_grad=True)


In [12]:
result = Model(test_input)

print("Result shape: ", result.shape)
print("Result: ", result)

Result shape:  torch.Size([1, 3, 5, 6])
Result:  tensor([[[[ 0.1000,  0.4000,  1.0000,  1.6000,  1.7000,  1.2000],
          [ 0.9000,  2.9000,  6.2000,  8.3000,  7.5000,  4.8000],
          [ 2.9000,  7.7000, 14.6000, 16.7000, 13.9000,  8.4000],
          [ 4.9000, 12.5000, 23.0000, 25.1000, 20.3000, 12.0000],
          [ 5.2000, 12.1000, 20.8000, 22.3000, 17.0000,  9.6000]],

         [[ 1.7000,  5.2000, 10.6000, 11.2000,  9.7000,  6.0000],
          [ 8.9000, 22.1000, 39.8000, 41.9000, 33.1000, 19.2000],
          [10.9000, 26.9000, 48.2000, 50.3000, 39.5000, 22.8000],
          [12.9000, 31.7000, 56.6000, 58.7000, 45.9000, 26.4000],
          [11.6000, 26.5000, 44.8000, 46.3000, 34.6000, 19.2000]],

         [[ 3.3000, 10.0000, 20.2000, 20.8000, 17.7000, 10.8000],
          [16.9000, 41.3000, 73.4000, 75.5000, 58.7000, 33.6000],
          [18.9000, 46.1000, 81.8000, 83.9000, 65.1000, 37.2000],
          [20.9000, 50.9000, 90.2000, 92.3000, 71.5000, 40.8000],
          [18.0000, 40.

<img src="./images/cnvtr1.png"><br>
<img src="./images/cnvtr2.png"><br>
<img src="./images/cnvtr3.png"><br>
<img src="./images/cnvtr4.png"><br>
<img src="./images/cnvtr5.png"><br>
<img src="./images/cnvtr6.png"><br>
<img src="./images/cnvtr7.png"><br>
<img src="./images/cnvtr8.png"><br>

Element Wise Addition: 행렬 덧셈

In [39]:
arr = (np.linspace(1, 16, 16)/10).reshape(4,4)
convtr1 = np.insert(arr, 4, 0, axis=1)
convtr1 = np.insert(convtr1, 4, 0, axis=1)
convtr1 = np.insert(convtr1, 4, 0, axis=0)

convtr2 = np.insert(arr*2, 0, 0, axis=1)
convtr2 = np.insert(convtr2, 5, 0, axis=1)
convtr2 = np.insert(convtr2, 4, 0, axis=0)

convtr3 = np.insert(arr*3, 0, 0, axis=1)
convtr3 = np.insert(convtr3, 0, 0, axis=1)
convtr3 = np.insert(convtr3, 4, 0, axis=0)

convtr4 = np.insert(arr*4, 4, 0, axis=1)
convtr4 = np.insert(convtr4, 4, 0, axis=1)
convtr4 = np.insert(convtr4, 0, 0, axis=0)

convtr5 = np.insert(arr*5, 0, 0, axis=1)
convtr5 = np.insert(convtr5, 5, 0, axis=1)
convtr5 = np.insert(convtr5, 0, 0, axis=0)

convtr6 = np.insert(arr*6, 0, 0, axis=1)
convtr6 = np.insert(convtr6, 0, 0, axis=1)
convtr6 = np.insert(convtr6, 0, 0, axis=0)

print(convtr1+convtr2+convtr3+convtr4+convtr5+convtr6)




[[ 0.1  0.4  1.   1.6  1.7  1.2]
 [ 0.9  2.9  6.2  8.3  7.5  4.8]
 [ 2.9  7.7 14.6 16.7 13.9  8.4]
 [ 4.9 12.5 23.  25.1 20.3 12. ]
 [ 5.2 12.1 20.8 22.3 17.   9.6]]


In [43]:
np.sum( (arr/13.6)*arr)

1.1